This is code I wrote to perform a MAC attack as part of my computer security class (CS 465).

In [3]:
# Bitwise Definitions and Printing
from ctypes import c_ushort 

def hex8(data):
    '''8bit int->hex converter'''
    return  '0x%02x' % (c_ushort(data).value)

def int16(data):
    '''16bit hex->int converter'''
    return c_ushort(int(data,16)).value

def printstuff(x, w=0, phrase=""):
    # Print in Hex
    if w==0:
        text = phrase
        for j in range(0, len(x)):
            text = text + " " + hex8(x[j])
        print text
    # Print in Ascii
    elif w==1:
        text = phrase
        for j in range(0, len(x)):
            text = text + str(unichr(x[j]))
        print text
    # Print in hex strings
    elif w==2:
        text = phrase
        for j in range(0, len(x)):
            text = text + hex8(x[j])[2:]
        print text
    else:
        print phrase,x

    print #For spacing purposes
        
def messageConvert(text):
    return lineConvert(text.encode("hex"))

def lineConvert(line):
    
    i = 0
    nums = []
    
    # Extract the numbers
    while i < len(line):
        text1 = line[i:i+2]
        nums.append(int16(text1))
        i = i + 2
        
    return nums


def arrayConvert(array):
    text = ""
    for j in range(0, len(array)):
        text = text + str(unichr(array[j]))
    return text


In [4]:
# Modified SHA1 code

import struct
import io

try:
    range = xrange
except NameError:
    pass

def _left_rotate(n, b):
    """Left rotate a 32-bit integer n by b bits."""
    return ((n << b) | (n >> (32 - b))) & 0xffffffff

def _process_chunk(chunk, h0, h1, h2, h3, h4):
    """Process a chunk of data and return the new digest variables."""
    assert len(chunk) == 64

    w = [0] * 80

    # Break chunk into sixteen 4-byte big-endian words w[i]
    for i in range(16):
        w[i] = struct.unpack(b'>I', chunk[i*4:i*4 + 4])[0]

    # Extend the sixteen 4-byte words into eighty 4-byte words
    for i in range(16, 80):
        w[i] = _left_rotate(w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16], 1)
    
    # Initialize hash value for this chunk
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4
    
    for i in range(80):
        if 0 <= i <= 19:
            # Use alternative 1 for f from FIPS PB 180-1 to avoid bitwise not
            f = d ^ (b & (c ^ d))
            k = 0x5A827999
        elif 20 <= i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i <= 59:
            f = (b & c) | (b & d) | (c & d) 
            k = 0x8F1BBCDC
        elif 60 <= i <= 79:
            f = b ^ c ^ d
            k = 0xCA62C1D6
    
        a, b, c, d, e = ((_left_rotate(a, 5) + f + e + k + w[i]) & 0xffffffff, 
                        a, _left_rotate(b, 30), c, d)
    
    # Add this chunk's hash to result so far
    h0 = (h0 + a) & 0xffffffff
    h1 = (h1 + b) & 0xffffffff 
    h2 = (h2 + c) & 0xffffffff
    h3 = (h3 + d) & 0xffffffff
    h4 = (h4 + e) & 0xffffffff
    
    print map(hex, (h0, h1, h2, h3, h4))

    return h0, h1, h2, h3, h4

class Sha1Hash(object):
    """A class that mimics that hashlib api and implements the SHA-1 algorithm."""

    name = 'python-sha1'
    digest_size = 20
    block_size = 64

    def __init__(self, h = None):
        # Initial digest variables
        
        if h == None:
            self._h = (
            0x67452301,
            0xEFCDAB89,
            0x98BADCFE,
            0x10325476,
            0xC3D2E1F0,
            )
        else:
            print map(hex, h)
            self._h = h

        # bytes object with 0 <= len < 64 used to store the end of the message
        # if the message length is not congruent to 64
        self._unprocessed = b''
        # Length in bytes of all data that has been processed so far
        self._message_byte_length = 0

    def update(self, arg):
        """Update the current digest.
        This may be called repeatedly, even after calling digest or hexdigest.
        
        Arguments:
            arg: bytes, bytearray, or BytesIO object to read from.
        """
        if isinstance(arg, (bytes, bytearray)):
            arg = io.BytesIO(arg)

        # Try to build a chunk out of the unprocessed data, if any
        chunk = self._unprocessed + arg.read(64 - len(self._unprocessed))
        
        # Read the rest of the data, 64 bytes at a time
        while len(chunk) == 64:
            self._h = _process_chunk(chunk, *self._h)
            self._message_byte_length += 64
            chunk = arg.read(64)

        self._unprocessed = chunk
        return self


    def digest(self):
        """Produce the final hash value (big-endian) as a bytes object"""
        return b''.join(struct.pack(b'>I', h) for h in self._produce_digest())

    def hexdigest(self):
        """Produce the final hash value (big-endian) as a hex string"""
        return '%08x%08x%08x%08x%08x' % self._produce_digest()

    def _produce_digest(self):
        """Return finalized digest variables for the data processed so far."""
        return self._h
        # Pre-processing:
        #message = self._unprocessed
        
        #message_byte_length = self._message_byte_length + len(message)
        
        # append the bit '1' to the message
        #message += b'\x80'
        
        # append 0 <= k < 512 bits '0', so that the resulting message length (in bytes)
        # is congruent to 56 (mod 64)
        #message += b'\x00' * ((56 - (message_byte_length + 1) % 64) % 64)
        
        # append length of message (before pre-processing), in bits, as 64-bit big-endian integer
        #message_bit_length = message_byte_length * 8
        #message += struct.pack(b'>Q', message_bit_length)

        # Process the final chunk
        # At this point, the length of the message is either 64 or 128 bytes.
        
        
        #h = _process_chunk(message[:64], *self._h)
        #if len(message) == 64:
        #    return h
        #return _process_chunk(message[64:], *h)


def sha1(data, IV = None):
    """SHA-1 Hashing Function
    A custom SHA-1 hashing function implemented entirely in Python.
    Arguments:
        data: A bytes or BytesIO object containing the input message to hash.
    Returns:
        A hex SHA-1 digest of the input message.
    """
    return Sha1Hash(IV).update(data).hexdigest()
    
    


In [5]:
# Modified SHA1 code

import struct
import io

try:
    range = xrange
except NameError:
    pass

def _left_rotate(n, b):
    """Left rotate a 32-bit integer n by b bits."""
    return ((n << b) | (n >> (32 - b))) & 0xffffffff

def _process_chunk(chunk, h0, h1, h2, h3, h4):
    """Process a chunk of data and return the new digest variables."""
    assert len(chunk) == 64

    w = [0] * 80

    # Break chunk into sixteen 4-byte big-endian words w[i]
    for i in range(16):
        w[i] = struct.unpack(b'>I', chunk[i*4:i*4 + 4])[0]

    # Extend the sixteen 4-byte words into eighty 4-byte words
    for i in range(16, 80):
        w[i] = _left_rotate(w[i-3] ^ w[i-8] ^ w[i-14] ^ w[i-16], 1)
    
    # Initialize hash value for this chunk
    a = h0
    b = h1
    c = h2
    d = h3
    e = h4
    
    for i in range(80):
        if 0 <= i <= 19:
            # Use alternative 1 for f from FIPS PB 180-1 to avoid bitwise not
            f = d ^ (b & (c ^ d))
            k = 0x5A827999
        elif 20 <= i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i <= 59:
            f = (b & c) | (b & d) | (c & d) 
            k = 0x8F1BBCDC
        elif 60 <= i <= 79:
            f = b ^ c ^ d
            k = 0xCA62C1D6
    
        a, b, c, d, e = ((_left_rotate(a, 5) + f + e + k + w[i]) & 0xffffffff, 
                        a, _left_rotate(b, 30), c, d)
    
    # Add this chunk's hash to result so far
    h0 = (h0 + a) & 0xffffffff
    h1 = (h1 + b) & 0xffffffff 
    h2 = (h2 + c) & 0xffffffff
    h3 = (h3 + d) & 0xffffffff
    h4 = (h4 + e) & 0xffffffff
    
    print map(hex, (h0, h1, h2, h3, h4))

    return h0, h1, h2, h3, h4

class Sha2Hash(object):
    """A class that mimics that hashlib api and implements the SHA-1 algorithm."""

    name = 'python-sha1'
    digest_size = 20
    block_size = 64

    def __init__(self, h = None):
        # Initial digest variables
        
        if h == None:
            self._h = (
            0x67452301,
            0xEFCDAB89,
            0x98BADCFE,
            0x10325476,
            0xC3D2E1F0,
            )
        else:
            print map(hex, h)
            self._h = h

        # bytes object with 0 <= len < 64 used to store the end of the message
        # if the message length is not congruent to 64
        self._unprocessed = b''
        # Length in bytes of all data that has been processed so far
        self._message_byte_length = 0

    def update(self, arg):
        """Update the current digest.
        This may be called repeatedly, even after calling digest or hexdigest.
        
        Arguments:
            arg: bytes, bytearray, or BytesIO object to read from.
        """
        if isinstance(arg, (bytes, bytearray)):
            arg = io.BytesIO(arg)

        # Try to build a chunk out of the unprocessed data, if any
        chunk = self._unprocessed + arg.read(64 - len(self._unprocessed))
        
        # Read the rest of the data, 64 bytes at a time
        while len(chunk) == 64:
            self._h = _process_chunk(chunk, *self._h)
            self._message_byte_length += 64
            chunk = arg.read(64)

        self._unprocessed = chunk
        return self


    def digest(self):
        """Produce the final hash value (big-endian) as a bytes object"""
        return b''.join(struct.pack(b'>I', h) for h in self._produce_digest())

    def hexdigest(self):
        """Produce the final hash value (big-endian) as a hex string"""
        return '%08x%08x%08x%08x%08x' % self._produce_digest()

    def _produce_digest(self):
        """Return finalized digest variables for the data processed so far."""
        return self._h
        # Pre-processing:
        message = self._unprocessed
        
        message_byte_length = self._message_byte_length + len(message)
        
        # append the bit '1' to the message
        message += b'\x80'
        
        # append 0 <= k < 512 bits '0', so that the resulting message length (in bytes)
        # is congruent to 56 (mod 64)
        message += b'\x00' * ((56 - (message_byte_length + 1) % 64) % 64)
        
        # append length of message (before pre-processing), in bits, as 64-bit big-endian integer
        message_bit_length = message_byte_length * 8
        message += struct.pack(b'>Q', message_bit_length)

        # Process the final chunk
        # At this point, the length of the message is either 64 or 128 bytes.
        
        
        h = _process_chunk(message[:64], *self._h)
        if len(message) == 64:
            return h
        return _process_chunk(message[64:], *h)


def sha2(data, IV = None):
    """SHA-1 Hashing Function
    A custom SHA-1 hashing function implemented entirely in Python.
    Arguments:
        data: A bytes or BytesIO object containing the input message to hash.
    Returns:
        A hex SHA-1 digest of the input message.
    """
    return Sha2Hash(IV).update(data).hexdigest()
    
    


In [6]:
# Implement Personal Mac Algorithm
import struct

def gethash(data):
    
    return sha1(data)

def packtext(data):
    result = ""
    for entry in data:
        s = struct.Struct("I")
        b = s.pack(entry)
        result = result + b

    return result
        

def sendMACmessage(text,key):
    # Append shared key with message
    start = key + text
    
    #start = padMessage(start)
    
    # Hash it to make MAC
    a = packtext(start)
    hashedmac = gethash(a)
    mac = lineConvert(hashedmac)
    
    # Send message with MAC
    message = text + mac

    return message

def verifyMACmessage(text,key):
    # Separate message from MAC
    message = text[:-20]
    mac = text[-20:]
    
    # Hash message with shared key
    a = packtext(key + message)
    hashedmac = gethash(a)
    made_mac = lineConvert(hashedmac)
    
    printstuff(made_mac,2,"Generated MAC: ")
    
    # Verify the generated MAC matches the given MAC
    if(all(x == made_mac[ind] for ind,x in enumerate(mac)) ):
        return True
    
    return False

# Implement Extension
def extendMessage(text,extension):
    # Separate message from MAC
    message = text[:-20]
    mac = text[-20:]

    #Pad the extension to fool the receiver
    extension = trickPadMessage(extension)
    
    # Hash padded extension with MAC as initilization vector
    new_mac = lineConvert(sha1(packtext(extension), prepareIV(mac)))
    
    # Return extended mession with new MAC
    extended_message = message+extension+new_mac
    
    return extended_message

def padMessage(message):
    
    message = arrayConvert(message)
    
    message_byte_length = len(message) + 16

    # append the bit '1' to the message
    message += b'\x80'

    # append 0 <= k < 512 bits '0', so that the resulting message length (in bytes)
    # is congruent to 56 (mod 64)
    message += b'\x00' * ((56 - (message_byte_length + 1) % 64) % 64)

    # append length of message (before pre-processing), in bits, as 64-bit big-endian integer
    message_bit_length = message_byte_length * 8
    message += struct.pack(b'>Q', message_bit_length)
    
    return messageConvert(message)
    

def trickPadMessage(message):
    
    message = arrayConvert(message)
    
    message_byte_length = len(message) + 128

    # append the bit '1' to the message
    message += b'\x80'

    # append 0 <= k < 512 bits '0', so that the resulting message length (in bytes)
    # is congruent to 56 (mod 64)
    message += b'\x00' * ((56 - (message_byte_length + 1) % 64) % 64)

    # append length of message (before pre-processing), in bits, as 64-bit big-endian integer
    message_bit_length = message_byte_length * 8
    message += struct.pack(b'>Q', message_bit_length)
    
    printstuff(messageConvert(message), 2, "Trick Pad: ")
    
    return messageConvert(message)
    
def prepareIV(mac):
    
    result = []
    
    for i in range(0,5):
        block = mac[4*i:4*i + 4]
        hexstrings = map(lambda x: hex8(x)[2:], block)
        hexvalue = "".join(hexstrings)
        result.append(int(hexvalue,16))
    
    return tuple(result)



    

In [7]:
# Testing

example_key = [ 0x2b, 0x28, 0xab, 0x09, 0x7e, 0xae, 0xf7, 0xcf,
                0x15, 0xd2, 0x15, 0x4f,0x16, 0xa6, 0x88, 0x3c]

example_text = messageConvert("No one has completed lab 2 so give them all a 0")

import hashlib
m = hashlib.sha1()
m.update("No one has completed lab 2 so give them all a 0")
print m.hexdigest()
print sha2("No one has completed lab 2 so give them all a 0")

# Test basic sending and verifying
mac_message = sendMACmessage(padMessage(example_text),example_key)

printstuff(mac_message[-20:],2, "Sent MAC: ")

print verifyMACmessage(mac_message, example_key)
print



#Make extension, send it, and verify it
extended_text = messageConvert(", except for David Hart. He gets full points.")

mac_message = sendMACmessage(padMessage(example_text) + trickPadMessage(extended_text), example_key)

printstuff(mac_message[-20:],2, "Sent MAC message: ")

print verifyMACmessage(mac_message, example_key)
print

#Send message, extend it, and verify it
mac_message = sendMACmessage(padMessage(example_text),example_key)

printstuff(mac_message[-20:],2, "Sent MAC message: ")

new_mac = extendMessage(mac_message, extended_text)

printstuff(new_mac[-20:],2, "Extended MAC: ")

print verifyMACmessage(new_mac, example_key)
print


7333ebf0fd0dc4d84f7f111c9378dc2398037928
67452301efcdab8998badcfe10325476c3d2e1f0
['0x7a38726c', '0x52c6e159', '0xe9ece6bd', '0xf4a71b25', '0xcaf221ac']
['0xe133a52e', '0xb60b1f95', '0x254d5652', '0xafddb6cd', '0x9b71088d']
['0x124d322', '0x4dff48f', '0xc44bf048', '0xd11846c2', '0xf0f6dbd1']
['0xd07b5938', '0x252eed7f', '0x993d6c6f', '0x94986064', '0xe4a149e3']
['0x42b1f837', '0x661201b', '0xe30e969', '0x1e7e91b4', '0x1207791c']
['0x617a75f9', '0x2ca334bd', '0x5673d078', '0xa08bab7a', '0x3ac695ab']
['0x5ab6a169', '0x5c4db5cb', '0x3c3ee05e', '0x1b2de5b3', '0xf29d198a']
['0xdd463a52', '0xb6c1dedd', '0xffc370f1', '0x80be6e19', '0xb62bcce5']
Sent MAC: dd463a52b6c1deddffc370f180be6e19b62bcce5

['0x7a38726c', '0x52c6e159', '0xe9ece6bd', '0xf4a71b25', '0xcaf221ac']
['0xe133a52e', '0xb60b1f95', '0x254d5652', '0xafddb6cd', '0x9b71088d']
['0x124d322', '0x4dff48f', '0xc44bf048', '0xd11846c2', '0xf0f6dbd1']
['0xd07b5938', '0x252eed7f', '0x993d6c6f', '0x94986064', '0xe4a149e3']
['0x42b1f837', '0x66

In [8]:
# Assignment Example
message = [0x4e, 0x6f, 0x20, 0x6f, 0x6e, 0x65, 0x20, 0x68, 0x61, 0x73, 0x20, 0x63, 0x6f, 0x6d, 0x70, 0x6c,0x65, 0x74, 0x65, 0x64, 0x20, 0x6c, 0x61, 0x62, 0x20, 0x32, 0x20, 0x73, 0x6f, 0x20, 0x67, 0x69, 0x76, 0x65, 0x20, 0x74, 0x68, 0x65, 0x6d, 0x20, 0x61, 0x6c, 0x6c, 0x20, 0x61, 0x20, 0x30];
last_hash = [0xf4,0xb6,0x45,0xe8,0x9f,0xaa,0xec,0x2f,0xf8,0xe4,0x43,0xc5,0x95,0x00,0x9c,0x16,0xdb,0xdf,0xba,0x4b]
extension = [0x2c,0x20,0x65,0x78,0x63,0x65,0x70,0x74,0x20,0x66,0x6f,0x72,0x20,0x44,0x61,0x76,0x69,0x64,0x20,0x48,0x61,0x72,0x74,0x2e]

printstuff(message,2, "Original message: ")
printstuff(padMessage(message),2,"Padded message: ")
new_mac = extendMessage(padMessage(message) + last_hash, extension)

printstuff(new_mac[:-20], 2, "Extended MAC message: ")

printstuff(new_mac[-20:], 2, "Extended MAC digest: ")





Extension: 2c2065786365707420666f7220446176696420486172742e

Original message: 4e6f206f6e652068617320636f6d706c65746564206c6162203220736f2067697665207468656d20616c6c20612030

Padded message: 4e6f206f6e652068617320636f6d706c65746564206c6162203220736f2067697665207468656d20616c6c2061203080000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001f8

Trick Pad: 2c2065786365707420666f7220446176696420486172742e800000000000000000000000000000000000000000000000000000000000000000000000000004c0

['0xf4b645e8', '0x9faaec2f', '0xf8e443c5', '0x95009c16', '0xdbdfba4b']
['0x9434cf6', '0xda6b2a00', '0xd7ed56cc', '0x6c33ab97', '0x84dfb28c']
['0xdfd50b3a', '0xb91610a3', '0xafb53ac1', '0x56f527c4', '0x6a0bcabd']
['0x78316eae', '0x5ee716ca', '0x94a769c3', '0x4a284253', '0xcd0fa25d']
['0x345f77be', '0x29251750', '0x3311a04b', '0x7cb8051d', '0x9fd0da3b']
Extended MAC message: 4e6f206f6e652068617320636f6d706c65746564206c6162203220736f20676976